<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/05_ML_04_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. MACHINE LEARNING

## 4-QADAM. Machine Learning

In [1]:
import pandas as pd
import numpy as np
import sklearn  # scikit-leaarn kutubxonasi

# Onlayn dataset joylashgan manzilni ko'rsatamiz
url = 'http://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true'
df = pd.read_csv(url)

- Ma'lumotlarni train va testga ajratamiz

In [2]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train= train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num=X_train.drop("ocean_proximity", axis=1)

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

#bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

# Pipeline da sonlar

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy="median")),
  ('attribs_adder', CombinedAttributesAdder()),
  ('std_scaler', StandardScaler()),
])

num_pipeline.fit_transform(X_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

#FullPipeline, Pipeline da matnlar bilan ishlash

In [5]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
  ("num", num_pipeline, num_attribs),
  ("cat", OneHotEncoder(), cat_attribs),
])

In [6]:
X_prepared = full_pipeline.fit_transform(X_train)

In [7]:
X_prepared[0:5,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.2117846 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.34218528,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.66165785,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
         0.78303162,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1

# LinearRegression modeli

In [8]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_prepared, y)

LinearRegression()

In [9]:
test_data = X_train.sample(10)
test_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
1596,-122.10,37.89,21.0,3282.0,653.0,1398.0,601.0,5.2079,NEAR BAY
1059,-121.89,39.76,15.0,10265.0,1860.0,4591.0,1906.0,3.0700,INLAND
15410,-117.24,33.21,9.0,2486.0,626.0,1938.0,525.0,2.1293,<1H OCEAN
20406,-118.87,34.20,26.0,1924.0,245.0,775.0,244.0,7.0010,<1H OCEAN
9355,-122.64,37.96,29.0,377.0,58.0,151.0,67.0,9.5551,NEAR OCEAN


In [10]:
test_labels = y.loc[test_data.index]
test_labels

1596     310300.0
1059     142600.0
15410    151400.0
20406    286800.0
9355     500001.0
19495    151400.0
1117      93400.0
13477    122700.0
11382    137300.0
19302    240900.0
Name: median_house_value, dtype: float64

In [11]:
test_data_prepared = full_pipeline.transform(test_data)
test_labels_predictions = lin_reg.predict(test_data_prepared)
test_labels_predictions

array([281581.95119548, 137834.79991002, 114247.29361622, 340060.54070108,
       458880.99376945, 208257.73779326,  77002.00483818, 126056.20890068,
       204147.76020182, 261237.08088946])

In [12]:
pd.DataFrame({"Actual": test_labels, "Predicted": test_labels_predictions})

,Actual,Predicted
1596,310300.0,281581.951195
1059,142600.0,137834.799910
15410,151400.0,114247.293616
20406,286800.0,340060.540701
9355,500001.0,458880.993769
19495,151400.0,208257.737793
1117,93400.0,77002.004838
13477,122700.0,126056.208901
11382,137300.0,204147.760202
19302,240900.0,261237.080889


# Modelni baholash

In [13]:
X_test = test_set.drop("median_house_value", axis=1)
X_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN


In [14]:
y_test = test_set["median_house_value"].copy()
y_test.head()

20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
Name: median_house_value, dtype: float64

In [15]:
X_test_prepared = full_pipeline.transform(X_test)

In [16]:
y_predicted = lin_reg.predict(X_test_prepared)

## Modelning o'rtacha absolyut xatoligini hisoblash

In [17]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)
mae

50898.7395349408

## Modelning o'rtacha kvadratik xatoligini hisoblash




In [18]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 72701.32600762138


# DecisionTree modeli

In [38]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_prepared, y)

DecisionTreeRegressor()

Modelni tekshiramiz

In [39]:
y_predicted = tree_reg.predict(X_test_prepared)

In [43]:
lin_mse = mean_squared_error(y_test, y_predicted)
lin_rmse = np.sqrt(lin_mse)
print("RMSE =", np.sqrt(lin_mse))

RMSE = 45353.39560867698


# Random Forest Modeli

In [19]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(X_prepared, y)

RandomForestRegressor()

In [20]:
y_predicted = forest_reg.predict(X_test_prepared)

In [21]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 50159.793869120185


# Cross-Validation modellarni aniqroq baholash

In [47]:
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"].copy()

X_prepared = full_pipeline.fit_transform(X)

Validation natijalarini ko'rsatish uchun sodda funksiya yasab olamiz

In [24]:
def display_scores(scores):
  print("Scores:", scores)
  print("Mean:", scores.mean())
  print("Standard deviation:", scores.std())

Cross-validation

In [44]:
from sklearn.model_selection import cross_val_score

##Linear Regression modeli xatoligi

In [53]:
lin_scores = cross_val_score(lin_reg, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

In [54]:
display_scores(np.sqrt(lin_rmse_scores))

Scores: [290.14421072 247.36921502 294.52268096 249.57310442 283.79087011
 262.52349726 229.13939878 301.50328014 278.70250991 232.25191861]
Mean: 266.95206859213806
Standard deviation: 25.004890824088356


## Decision Tree modeli xatoligi

In [55]:
dec_scores = cross_val_score(tree_reg, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
dec_rmse_scores = np.sqrt(-dec_scores)

In [56]:
display_scores(dec_rmse_scores)

Scores: [112808.01804578  70116.59117684  82481.35753817  74239.73556172
  90185.14744058  78884.73648467  67864.50919335 100848.40663927
  95662.92066012  76094.45781786]
Mean: 84918.58805583793
Standard deviation: 13852.610208946458


## Forest Regression modeli xatoligi

In [51]:
scores = cross_val_score(forest_reg, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

In [52]:
display_scores(forest_rmse_scores)

Scores: [97544.80962584 47425.4512783  64980.59031521 56776.27471768
 60875.97187434 59627.47593391 47219.445561   79706.80087299
 74317.54458304 49009.49553688]
Mean: 63748.386029918955
Standard deviation: 15299.727632370108


# Modelni saqlash

### pickle

In [33]:
import pickle

filename = "lin_reg.pkl"
with open(filename, "wb") as file:  # faylga istalgan nom berish mumkin
  pickle.dump(lin_reg, file)

# with open("forest_reg.pkl", "wb") as file:
#   pickle.dump(forest_reg, file)

### joblib

In [29]:
import joblib

joblib.dump(lin_reg, "lin_reg.jbl")
joblib.dump(forest_reg, "forest_reg.jbl")

['forest_reg.jbl']

# Saqlangan Modelni o'qish

### pickle

In [ ]:
with open('forest_reg.pkl', "rb") as file:
  model = pickle.load(file)

### joblib

In [36]:
filename = "forest_reg.jbl"
model_reg = joblib.load(filename)

In [37]:
scores = cross_val_score(model_reg, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [77441.01164809 63969.38952083 61288.74849827 81289.78586544
 62237.7397511 ]
Mean: 69245.335056744
Standard deviation: 8396.275043150008


## Pipline ni saqlab olamiz

In [57]:
filename = 'pipeline.jbl'
joblib.dump(full_pipeline, filename)

['pipeline.jbl']